# Project: AI Model Experimentation

This notebook goal is to test differente solutions to the conection between the AI model and the ToDo app in order to receive a description of the information in the entity "taks". 

## 1. Using llama_index to connect the LLM to the data base

In [67]:
# Import necessary libraries
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine, MetaData, Table
from llama_index.core import SQLDatabase, VectorStoreIndex
from llama_index.core.query_engine import NLSQLTableQueryEngine

from llama_index.core.llms import CustomLLM, LLMMetadata
from llama_index.core.base.llms.types import CompletionResponse
import requests
from typing import Any, Iterator, Optional

In [68]:
# Load environment variables
load_dotenv()

True

In [5]:
# Contection test to CodeGPT API as the documentation suggests

api_key = os.getenv("OPENAI_API_KEY")
api_base = os.getenv("CODEGPT_API_BASE")
agent_id = os.getenv("AGENT_ID")
organization_id = os.getenv("ORGANIZATION_ID")


payload = {
    "agentId": f"{agent_id}",
    "messages": [{"content":"hola",
                  "role": "user"}]
}
headers = {
    "accept": "application/json",
    "CodeGPT-Org-Id": f"{organization_id}",
    "content-type": "application/json",
    "authorization": f"Bearer {api_key}"
}

response = requests.post(f"{api_base}", json=payload, headers=headers)

print(response.text)

¡Hola! ¿En qué puedo ayudarte hoy?


In [6]:
# Initialize llm connection
class CodeGPTLLM(CustomLLM):
    api_key: Optional[str] = None
    api_base: Optional[str] = None
    agent_id: Optional[str] = None
    organization_id: Optional[str] = None

    def __init__(self):                
        api_key = os.getenv("OPENAI_API_KEY")
        api_base = os.getenv("CODEGPT_API_BASE")
        agent_id = os.getenv("AGENT_ID")
        organization_id = os.getenv("ORGANIZATION_ID")
        super().__init__(
            api_key=api_key,
            api_base=api_base,
            agent_id=agent_id,
            organization_id=organization_id)
        
        
    @property
    def metadata(self) -> LLMMetadata:
        return LLMMetadata(
            context_window=4096,
            num_output=256,
            model_name="codegpt-custom",
            is_chat_model=True,  # Set to True if your model supports chat format
            is_function_calling_model=False
        )   

    def complete(self, prompt: str, **kwargs:Any) -> CompletionResponse:
        try: 
            payload = {
                "agentId": f"{self.agent_id}",                      
                "messages": [{
                        "content":  f"{prompt}" ,
                        "role": "user"
                        }]
                }
            
            headers = {
                "accept": "application/json",
                "CodeGPT-Org-Id": f"{self.organization_id}",
                "content-type": "application/json",
                "authorization": f"Bearer {self.api_key}"
            }


            response = requests.post(
                f"{self.api_base}",
                json=payload,
                headers=headers
            )
            
            return CompletionResponse(text=response.text)
        except Exception as e:
            print(f"Error in CodeGPTLLM call: {e}")
            return CompletionResponse(text="Error in generating response")
        
    def stream_complete(self, prompt: str, **kwargs: Any) -> Iterator[CompletionResponse]:
        # For non-streaming implementation, just yield the complete response
        response = self.complete(prompt, **kwargs)
        yield response    


In [69]:
llm = CodeGPTLLM()

In [8]:
call = llm.complete("What is the capital of France?")
print(call.text)

The capital of France is Paris.


In [71]:
# set up database connection
db_path = os.path.join("..", "database", "todo.db")
print(f"Connecting to database at: {db_path}")
engine = create_engine(f"sqlite:///{db_path}")
metadata = MetaData()
metadata.reflect(bind=engine)

Connecting to database at: ../database/todo.db


In [72]:
# get the information about the table "tasks"
tasks_table = Table("tasks", metadata, autoload_with=engine)
print(tasks_table.columns.keys())

['id', 'task', 'description', 'creation_time', 'end_time', 'finished']


In [73]:
# Configurar SQLDatabase para LlamaIndex
sql_database = SQLDatabase(engine, include_tables=["tasks"])

In [16]:
# create a Query Engine
query_engine = NLSQLTableQueryEngine(
    sql_database=sql_database,
    tables=["tasks"],
    llm=llm,
    synthesize_response=True
)

In [17]:
# create the funtion to query the database
def query_tasks(question: str) -> str:

    response = query_engine.query(question)
    return str(response)

In [30]:
questions = [
        "summarize the tasks",
        "give me a motivational phrase to do tasks",
        "give me a list of tasks, including the task name, description, creation time, end time and status"
    ]

In [32]:
for q in questions:
        print(f"Pregunta: {q}")
        respuesta = query_tasks(q)
        print(f"Respuesta: {respuesta}\n")

Pregunta: summarize the tasks
Respuesta: You have 4 tasks in total, 2 complete (50%). 2 still pending. Focus on the remaining tasks.

Pregunta: give me a motivational phrase to do tasks
Respuesta: "You have 2 tasks completed. Keep up the good work! 'The future depends on what you do today.' - Mahatma Gandhi"

Pregunta: give me a list of tasks, including the task name, description, creation time, end time and status
Respuesta: Here is the list of tasks:

1. Task: Complete project documentation
   - Description: Write comprehensive documentation for the ToDo API
   - Creation Time: 2025-07-06 12:39:35
   - End Time: None
   - Status: Pending

2. Task: Buy milk and eggs
   - Description: I need 3 liters
   - Creation Time: 2025-07-06 12:41:34
   - End Time: 2025-07-06 11:19:19
   - Status: Completed

3. Task: Dummy task to delete
   - Description: Please work
   - Creation Time: 2025-07-06 13:45:59
   - End Time: None
   - Status: Pending

4. Task: Finish backend
   - Description: AI rout

## 2 - Create a promt with the DB information from a query it self

In [61]:
api_key = os.getenv("OPENAI_API_KEY")
api_base = os.getenv("CODEGPT_API_BASE")
agent_id = os.getenv("AGENT_ID")
organization_id = os.getenv("ORGANIZATION_ID")

def call_codegpt_api(promt: str) -> str:
    payload = {
        "agentId": f"{agent_id}",
        "messages": [{"content":f"{promt}",
                    "role": "user"}]
    }
    headers = {
        "accept": "application/json",
        "CodeGPT-Org-Id": f"{organization_id}",
        "content-type": "application/json",
        "authorization": f"Bearer {api_key}"
    }

    response = requests.post(f"{api_base}", json=payload, headers=headers)

    return response.text

In [62]:
GETALL_API = "http://127.0.0.1:8070/crud/getall"

response = requests.get(GETALL_API)
if response.status_code == 200:
    data = response.json()
    print(data)
else:
    print("Failed to retrieve data from the API")
 

[[1, 'Complete project documentation', 'Write comprehensive documentation for the ToDo API', '2025-07-06 12:39:35', None, 'N'], [2, 'buy milk and eggs', 'I need 3 litters', '2025-07-06 12:41:34', '2025-07-06 11:19:19', 'Y'], [5, 'dumy task to delete', 'please work', '2025-07-06 13:45:59', None, 'N'], [8, 'finish backend', 'AI router', '2025-07-06 19:57:54', None, 'N']]


In [63]:
prompt = f"Give a sumary of the the next tasks: {data}, add a motivational phrase to do tasks"

In [64]:
response = call_codegpt_api(prompt)
print(response)

You have 4 tasks in total, 1 complete (25%). 3 still pending. "The future depends on what you do today." - Mahatma Gandhi


In [65]:
prompt = f"Give a list of the the next tasks: {data}, I want to know task name, description, creation time, endtime if finished and status."

In [66]:
response = call_codegpt_api(prompt)
print(response)

Here is the list of your tasks:

1. **Task Name:** Complete project documentation
   - **Description:** Write comprehensive documentation for the ToDo API
   - **Creation Time:** 2025-07-06 12:39:35
   - **End Time:** None
   - **Status:** Not Finished

2. **Task Name:** buy milk and eggs
   - **Description:** I need 3 litters
   - **Creation Time:** 2025-07-06 12:41:34
   - **End Time:** 2025-07-06 11:19:19
   - **Status:** Finished

3. **Task Name:** dumy task to delete
   - **Description:** please work
   - **Creation Time:** 2025-07-06 13:45:59
   - **End Time:** None
   - **Status:** Not Finished

4. **Task Name:** finish backend
   - **Description:** AI router
   - **Creation Time:** 2025-07-06 19:57:54
   - **End Time:** None
   - **Status:** Not Finished
